In [4]:
import os
import shutil
import re
import subprocess
import urllib
import zipfile
import requests
import sys


class CantGetLatestChromiumVersionError(Exception):
    """Happens when regex failed"""

class ChromiumInstallationFailedException(Exception):
    """
    Happens when deb package not installed
    Check the downloaded chroumium deb file
    """

class CantGetChromeDriverError(Exception):
    """Happens when regex failed"""

main_url = "http://packages.linuxmint.com/pool/upstream/c/chromium/"
work_dir = "/content"

def get_chromium_latest_version() -> str:
    # A request to packages.linuxmint.com for getting latest version of chromium
    # e.g. "chromium_121.0.6167.160~linuxmint1+virginia_amd64.deb"
    r = requests.get(main_url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="(chromium_[^"]+linuxmint1%2Bvirginia_amd64.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise CantGetLatestChromiumVersionError("Failed to get latest chromium version!")
    return latest_version

def install_chromium(latest_version: str, deb_file: str, quiet: bool):
    # Full url of deb file
    url = f"{main_url}{latest_version}"

    # Download deb file
    if quiet:
        command = f"wget -q -O {work_dir}/{deb_file} {url}"
    else:
        command = f"wget -O {work_dir}/{deb_file} {url}"
    print(f"Downloading: {deb_file}")
    # os.system(command)
    !$command

    # Install deb file
    if quiet:
        command = f"apt-get install {work_dir}/{deb_file} >> apt.log"
    else:
        command = f"apt-get install {work_dir}/{deb_file}"
    print(f"Installing: {deb_file}")
    # os.system(command)
    !$command

def check_chromium_installation(deb_file: str):
    try:
        subprocess.call(["/usr/bin/chromium"])
        print("Chromium installation successfull.\n")
        # If installation successfull we can remove deb file
        # Delete deb file from disk
        os.remove(f"{work_dir}/{deb_file}")
    except FileNotFoundError:
        raise ChromiumInstallationFailedException("Chromium Installation Failed!")

def get_chromedriver_url(deb_file: str) -> str:
    # Get content of crhomedriver page
    url = "https://googlechromelabs.github.io/chrome-for-testing/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Get chromium version from deb file's name
    version_number = deb_file.split("chromium_")[-1].split(".")[0]

    # Example: https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/121.0.6167.85/linux64/chromedriver-linux64.zip
    pattern = f'https://[^<]+/{version_number}[^<]+/linux64/chromedriver-linux64.zip'
    # Find latest version
    chromedriver_url_search = re.search(pattern, text)
    if chromedriver_url_search:
        chromedriver_url = chromedriver_url_search.group()
        return chromedriver_url
    else:
        raise CantGetChromeDriverError("Failed to get chromedriver!")

def install_chromedriver(deb_file: str, quiet: bool):
    url = get_chromedriver_url(deb_file)
    file_name = url.split("/")[-1]
    # Download chromedriver
    chromedriver_zip = f"{work_dir}/{file_name}"
    if quiet:
        command = f"wget -q -O {chromedriver_zip} {url}"
    else:
        command = f"wget -O {chromedriver_zip} {url}"
    print(f"Downloading: {file_name}")
    # os.system(command)
    !$command

    # Extract chromedriver from zip
    with zipfile.ZipFile(chromedriver_zip) as zpf:
        zpf.extract(member="chromedriver-linux64/chromedriver", path=work_dir)

    # Remove chromedriver-linux64.zip file
    os.remove(chromedriver_zip)

    # Move extracted chromedriver binary file to /usr/bin directory
    source = f"{work_dir}/chromedriver-linux64/chromedriver"
    destination = "/usr/bin/chromedriver"
    os.rename(source, destination)

    # Make chromedriver binary executable
    os.system(f"chmod +x {destination}")

    # Remove empty chromedriver-linux64 folder
    shutil.rmtree(f"{work_dir}/chromedriver-linux64")

    print("Chromedriver installed")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium from linux mint packages site
    latest_version = get_chromium_latest_version()
    # Name of the deb file
    deb_file = urllib.parse.unquote(latest_version, "utf-8")
    # Download and install chromium for ubuntu 22.04
    install_chromium(latest_version, deb_file, quiet)
    # Check if installation succesfull
    check_chromium_installation(deb_file)
    # Install chromedriver
    install_chromedriver(deb_file, quiet)
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium_131.0.6778.85~linuxmint1+virginia_amd64.deb
Installing: chromium_131.0.6778.85~linuxmint1+virginia_amd64.deb


'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'apt-get' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


ChromiumInstallationFailedException: Chromium Installation Failed!

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

# 瀏覽器選項
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
# 輸入中文名稱進行導航

def navigate_and_extract(driver, location_name, xpath_mapping):
    try:
        # 展開父選單
        parent_menu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.menu-item-has-children > a'))
        )
        driver.execute_script("arguments[0].click();", parent_menu)

        # 點擊對應的選項
        location_xpath = xpath_mapping.get(location_name)
        if not location_xpath:
            raise ValueError(f"找不到名稱『{location_name}』的對應 XPATH")

        location_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, location_xpath))
        )
        driver.execute_script("arguments[0].click();", location_element)

        # 初始化頁數
        page_number = 1
        all_data = []

        while True:
            # 等待頁面加載完成
            WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

            # 確認是否進入最後一頁
            if "Oops! That page can’t be found." in driver.page_source:
                print("已到最後一頁，停止抓取。")
                break

            # 抓取頁面內所有文章的資料
            articles = driver.find_elements(By.CSS_SELECTOR, "article")

            for article in articles:
                try:
                    # 抓取標題
                    title_element = article.find_element(By.CSS_SELECTOR, "h2 a")
                    title = title_element.text
                    link = title_element.get_attribute("href")

                    # 抓取內容
                    try:
                        content_element = article.find_elements(By.CSS_SELECTOR, "p, font[style*='vertical-align']")
                        content = "\n".join([e.text for e in content_element if e.text.strip()]) if content_element else "無內容"
                    except Exception:
                        content = "無內容"

                    # 抓取圖片
                    try:
                        image_element = article.find_element(By.CSS_SELECTOR, "img")
                        image = image_element.get_attribute("src")
                    except Exception:
                        image = "無圖片"

                    all_data.append({
                        "title": title,
                        "link": link,
                        "content": content,
                        "image": image
                    })
                except Exception as e:
                    print(f"跳過一篇文章，錯誤: {e}")

            # 嘗試進入下一頁
            try:
                page_number += 1
                next_page_url = f"/page/{page_number}/"
                driver.get(driver.current_url.rstrip('/') + next_page_url)
            except Exception as e:
                print(f"無法進入下一頁: {e}")
                break

        return all_data
    except Exception as e:
        raise RuntimeError(f"導航到『{location_name}』時失敗: {e}")

# 主程式
if __name__ == "__main__":
    driver = webdriver.Chrome(options=options)
    url = "https://markandhazyl.com/"
    driver.get(url)

    # 測試用 XPATH 映射表
    xpath_mapping = {
            "日本": "//*[@id=\"menu-item-104507\"]/a",
            "北海道": "//*[@id=\"menu-item-104511\"]/a",
            "東北地區": "//*[@id=\"menu-item-104513\"]/a",
            "關東地區": "//*[@id=\"menu-item-104516\"]/a",
            "中部地區": "//*[@id=\"menu-item-104509\"]/a",
            "近畿地區": "//*[@id=\"menu-item-104515\"]/a",
            "中國地區": "//*[@id=\"menu-item-104508\"]/a",
            "四國地區": "//*[@id=\"menu-item-104512\"]/a",
            "九州地區": "//*[@id=\"menu-item-104510\"]/a",
            "沖繩": "//*[@id=\"menu-item-104514\"]/a",
            "台灣": "//*[@id=\"menu-item-104391\"]/a",
            "台北": "//*[@id=\"menu-item-104392\"]/a",
            "桃園": "//*[@id=\"menu-item-104395\"]/a",
            "新竹": "//*[@id=\"menu-item-104394\"]/a",
            "宜蘭": "//*[@id=\"menu-item-104393\"]/a",
            "苗栗": "//*[@id=\"menu-item-104396\"]/a",
            "台中": "//*[@id=\"menu-item-104397\"]/a",
            "彰化": "//*[@id=\"menu-item-104398\"]/a",
            "雲林": "//*[@id=\"menu-item-104400\"]/a",
            "南投": "//*[@id=\"menu-item-104399\"]/a",
            "花蓮": "//*[@id=\"menu-item-104401\"]/a",
            "嘉義": "//*[@id=\"menu-item-104403\"]/a",
            "台南": "//*[@id=\"menu-item-104402\"]/a",
            "高雄": "//*[@id=\"menu-item-104406\"]/a",
            "屏東": "//*[@id=\"menu-item-104405\"]/a",
            "台東": "//*[@id=\"menu-item-104404\"]/a",
            "離島": "//*[@id=\"menu-item-104407\"]/a",
            "韓國": "//*[@id=\"menu-item-104551\"]/a",
            "首爾": "//*[@id=\"menu-item-104572\"]/a",
            "釜山": "//*[@id=\"menu-item-104571\"]/a",
            "大邱": "//*[@id=\"menu-item-104569\"]/a",
            "濟州島": "//*[@id=\"menu-item-104570\"]/a",
            "泰國": "//*[@id=\"menu-item-104679\"]/a",
            "曼谷": "//*[@id=\"menu-item-104674\"]/a",
            "普吉島": "//*[@id=\"menu-item-104678\"]/a",
            "芭達雅": "//*[@id=\"menu-item-104681\"]/a",
            "清邁": "//*[@id=\"menu-item-104683\"]/a",
            "華欣": "//*[@id=\"menu-item-104685\"]/a",
            "亞洲": "//*[@id=\"menu-item-104554\"]/a",
            "馬來西亞": "//*[@id=\"menu-item-104553\"]/a",
            "新加坡": "//*[@id=\"menu-item-104557\"]/a",
            "越南": "//*[@id=\"menu-item-104550\"]/a",
            "印尼": "//*[@id=\"menu-item-104556\"]/a",
            "菲律賓": "//*[@id=\"menu-item-104549\"]/a",
            "中國": "//*[@id=\"menu-item-104555\"]/a",
            "香港": "//*[@id=\"menu-item-104552\"]/a",
            "澳門": "//*[@id=\"menu-item-104548\"]/a",
            "紐澳": "//*[@id=\"menu-item-104413\"]/a",
            "紐西蘭": "//*[@id=\"menu-item-104415\"]/a",
            "澳洲": "//*[@id=\"menu-item-104414\"]/a",
            "歐洲": "//*[@id=\"menu-item-104417\"]/a",
            "英國": "//*[@id=\"menu-item-104423\"]/a",
            "義大利": "//*[@id=\"menu-item-104422\"]/a",
            "法國": "//*[@id=\"menu-item-104420\"]/a",
            "德國": "//*[@id=\"menu-item-104419\"]/a",
            "瑞士": "//*[@id=\"menu-item-104421\"]/a",
            "冰島": "//*[@id=\"menu-item-104418\"]/a",
    }

    try:
        # 從命令列參數接收輸入地區名稱


        location_name = "日本"

        # 初始化瀏覽器驅動
        result = navigate_and_extract(driver, location_name, xpath_mapping)

        # 返回爬取結果作為 JSON
        print(json.dumps(result, ensure_ascii=False, indent=4))

    except Exception as e:
        # 返回錯誤訊息
        print(json.dumps({"error": f"操作失敗: {e}"}))

    finally:
        if 'driver' in locals():
            driver.quit()

已到最後一頁，停止抓取。
[
    {
        "title": "TOP7琵琶湖住宿推薦【2024】泡溫泉欣賞超美琵琶湖畔景",
        "link": "https://markandhazyl.com/where-to-stay-in-lake-biwa/",
        "content": "琵琶湖住宿推薦這幾間琵琶湖飯店、琵琶湖溫泉飯店！琵琶湖位於日本滋賀縣，距離京都市約30公里。這座湖泊形狀如琵琶，是日本歷史最悠久、面積最大的淡水湖。湖泊周圍環繞著壯麗的山脈和風景如畫的小鎮，形成了一個豐富的生態系統和迷人的自然景觀。在琵琶湖附近，可以選擇各式各樣的住宿選項，從豪華度假村到溫馨民宿應有盡有！這些旅館不僅提\n延伸閱讀\nTOP7琵琶湖住宿推薦【2024】泡溫泉欣賞超美琵琶湖畔景",
        "image": "https://markandhazyl.com/wp-content/uploads/2024/11/%E7%90%B5%E7%90%B6%E6%B9%96%E4%BD%8F%E5%AE%BF%E6%8E%A8%E8%96%A6-768x614.webp"
    },
    {
        "title": "北海道新千歲機場必買伴手禮【2024】好吃好買好逛全攻略",
        "link": "https://markandhazyl.com/what-to-buy-in-hokkaido-new-chitose-airport/",
        "content": "新千歲機場在經過兩次翻新後，進駐了超過180個店家聚集在此，陪伴每位觀光客度過候機時間，堪稱全日本最好逛又最好買的機場。\n機場是搭機回家前最後可以把握購買伴手禮的地方，這裡伴手禮的店大多聚集在國內線航廈二樓，知名的白色戀人、Royce和六花亭等都在這裡，跟著以下必買攻略，直接在機場把伴手禮買好買滿，讓你省去在路上扛著大\n延伸閱讀\n北海道新千歲機場必買伴手禮【2024】好吃好買好逛全攻略",
        "image": "https://markandhazyl.com/wp-content/uploads/2024/09/%E5%8C%97%E6%B5%B7%E9%81%93%E6%96%B0%E5%8D%83%E6